### Učitavanje GenBank fajla i parsiranje zapisa
Fajl `data/NC_000913.3.gb` preuzet sa https://www.ncbi.nlm.nih.gov/nuccore/NC_000913.3?report=gbwithparts&log$=seqview

In [16]:
from Bio import SeqIO
data = SeqIO.parse("data/NC_000913.3.gb", "genbank")

In [17]:
records = [record for record in data]

In [18]:
print(f'Total records: {len(records)}')

Total records: 1


In [19]:
record = records[0]
print(record)

ID: NC_000913.3
Name: NC_000913
Description: Escherichia coli str. K-12 substr. MG1655, complete genome
Database cross-references: BioProject:PRJNA57779, BioSample:SAMN02604091
Number of features: 9285
/molecule_type=DNA
/topology=circular
/data_file_division=CON
/date=17-JUL-2025
/accessions=['NC_000913']
/sequence_version=3
/gi=556503834
/keywords=['RefSeq']
/source=Escherichia coli str. K-12 substr. MG1655
/organism=Escherichia coli str. K-12 substr. MG1655
/taxonomy=['Bacteria', 'Pseudomonadati', 'Pseudomonadota', 'Gammaproteobacteria', 'Enterobacterales', 'Enterobacteriaceae', 'Escherichia']
/references=[Reference(title='Escherichia coli K-12: a cooperatively developed annotation snapshot--2005', ...), Reference(title='Highly accurate genome sequences of Escherichia coli K-12 strains MG1655 and W3110', ...), Reference(title='The complete genome sequence of Escherichia coli K-12', ...), Reference(title='Workshop on Annotation of Escherichia coli K-12', ...), Reference(title='ASAP: 

In [21]:
sequence = record.seq
print(f'Dužina sekvence: {len(sequence)}')

Dužina sekvence: 4641652


In [23]:
CDS = [feature for feature in record.features if feature.type == 'CDS']
print(f'Broj CDS-ova: {len(CDS)}')

Broj CDS-ova: 4318


### Verifikacija podataka
Provera da li su svi CDS-ovi validni:
- dužina nukleotidnog niza je deljiva sa 3
- koristi se standardna translaciona tabela (11)
- postoji anotirana aminokiselinska sekvenca
- translacija prati striktna CDS pravila (npr. nema unutrašnjih stop kodona)
- prevedena aminokiselinska sekvenca se slaže sa anotiranom
- postoji UniProt identifikator zbog daljih analiza (https://iupred3.elte.hu/)

In [24]:
def get_cds_accession(cds):
    # Vraca UniProtKB/Swiss-Prot ili UniProtKB/TrEMBL identifikator ako postoji
    for db_xref in cds.qualifiers.get('db_xref', []):
        if db_xref.startswith('UniProtKB/Swiss-Prot:') or db_xref.startswith('UniProtKB/TrEMBL:'):
            return db_xref.split(':')[1]

In [25]:
verified_cds = []
corrupted_cds = []
for cds in CDS:
    extracted_cds = cds.extract(record)
    nucleotide_seq = extracted_cds.seq
    nucleotide_sequence_length = len(nucleotide_seq)
    protein_id = cds.qualifiers.get('protein_id', [None])[0]
    cds_locus_tag = cds.qualifiers.get('locus_tag', [None])[0]
    if nucleotide_sequence_length % 3 != 0:
        corrupted_cds.append(cds)
        print(f'Neispravan CDS na lokusu {cds_locus_tag} za protein {protein_id}:  Dužina sekvence {nucleotide_sequence_length} nije deljiva sa 3!')
        continue

    translation_table = cds.qualifiers.get('transl_table', [None])[0]

    if translation_table != '11':
        corrupted_cds.append(cds)
        print(f'Neispravan CDS na lokusu {cds_locus_tag} za protein {protein_id}:  Ne koristi standardnu translacionu tabelu: {translation_table}')
        continue

    aa_annotated = cds.qualifiers.get('translation', [None])[0]
    if aa_annotated is None:
        corrupted_cds.append(cds)
        print(f'Neispravan CDS na lokusu {cds_locus_tag} za protein {protein_id}:  Ne postoji informacija o sekvenci amino kiseline!')
        continue
    try:
        aa_translated = nucleotide_seq.translate(table=translation_table, to_stop=True, cds=True)
    except Exception as err:
        corrupted_cds.append(cds)
        print(f'Neispravan CDS na lokusu {cds_locus_tag} za protein {protein_id}:  Greska prilikom translacije: {err}')
        continue

    aa_translated = nucleotide_seq.translate(table=translation_table, to_stop=True, cds=True)
    if str(aa_translated) != aa_annotated:
        corrupted_cds.append(cds)
        print(f'Neispravan CDS na lokusu {cds_locus_tag} za protein {protein_id}:  Translacije se ne podudaraju!')
        continue

    if get_cds_accession(cds) is None:
        corrupted_cds.append(cds)
        print(f'Neispravan CDS na lokusu {cds_locus_tag} za protein {protein_id}:  Ne postoji UniProt identifikator!')
        continue

    verified_cds.append(cds)

Neispravan CDS na lokusu b0149 za protein NP_414691.1:  Ne postoji UniProt identifikator!
Neispravan CDS na lokusu b0149 za protein YP_010051172.1:  Ne postoji UniProt identifikator!
Neispravan CDS na lokusu b0240 za protein None:  Ne postoji informacija o sekvenci amino kiseline!
Neispravan CDS na lokusu b4710 za protein YP_009518739.1:  Ne postoji UniProt identifikator!
Neispravan CDS na lokusu b4709 za protein YP_009518740.1:  Ne postoji UniProt identifikator!
Neispravan CDS na lokusu b4587 za protein None:  Ne postoji informacija o sekvenci amino kiseline!
Neispravan CDS na lokusu b4845 za protein YP_013606033.1:  Ne postoji UniProt identifikator!
Neispravan CDS na lokusu b4579 za protein None:  Ne postoji informacija o sekvenci amino kiseline!
Neispravan CDS na lokusu b4580 za protein None:  Ne postoji informacija o sekvenci amino kiseline!
Neispravan CDS na lokusu b0470 za protein NP_415003.1:  Ne postoji UniProt identifikator!
Neispravan CDS na lokusu b0484 za protein NP_415017.

In [26]:
print(f'Ukupan broj ispravnih CDS-ova: {len(verified_cds)}')
print(f'Ukupan broj CDS-ova sa greškom: {len(corrupted_cds)}')

Ukupan broj ispravnih CDS-ova: 4277
Ukupan broj CDS-ova sa greškom: 41


### Prikupljanje podataka o neuređenosti delova proteina

#### IUpred3 (https://iupred3.elte.hu/) predvidjanje neuređenosti na osnovu uniprot identifikatora putem API-ja

In [28]:
import requests

def get_disorder_data_iupred3(accession, iupred_type="long"):
    url = f"http://iupred3.elte.hu/iupred3/{iupred_type}/{accession}.json"
    try:
        response = requests.get(url)
        response.raise_for_status()
        result = response.json()

        disorder_scores = result['iupred2']

        return disorder_scores
    except requests.exceptions.RequestException as e:
        print(f"Request error for {accession}: {e}")
        return []
    except KeyError as e:
        print(f"JSON parsing error for {accession}: {e}")
        return []

#### IsUnstruct (http://bioinfo.protres.ru/IsUnstruct/) predvidjanje neuređenosti na osnovu aminokiselinske sekvence putem lokalno instaliranog softvera

In [49]:
import subprocess
import tempfile
import os

def windows_to_wsl_path(windows_path):
    """Convert Windows path to WSL path format"""
    # Convert C:\path\to\file to /mnt/c/path/to/file
    if windows_path[1:3] == ':\\':
        drive_letter = windows_path[0].lower()
        rest_of_path = windows_path[3:].replace('\\', '/')
        return f'/mnt/{drive_letter}/{rest_of_path}'
    return windows_path.replace('\\', '/')

def get_disorder_probabilities_isunstruct(sequence):

    if not sequence:
        return []

    # Create temporary FASTA file
    with tempfile.NamedTemporaryFile(mode='w+', delete=False, suffix='.fasta') as temp_file:
        temp_file.write(f">seq\n{sequence}\n")
        temp_file_path = temp_file.name

    try:
        # Convert Windows path to WSL format
        binary_path = os.path.join(os.getcwd(), "software", "is_unstruct", "IsUnstruct")
        wsl_binary_path = binary_path.replace('C:\\', '/mnt/c/').replace('\\', '/')
        wsl_temp_path = temp_file_path.replace('C:\\', '/mnt/c/').replace('\\', '/')

        # Run IsUnstruct
        result = subprocess.run(['wsl', wsl_binary_path, '-long_disp', '1', '-file_save', '0', wsl_temp_path],
                              capture_output=True, text=True, check=True)

        # Extract probabilities
        probabilities = []
        for line in result.stdout.strip().split('\n'):
            if not line.startswith('#') and line.strip():
                parts = line.strip().split()
                if len(parts) >= 4:
                    probabilities.append(float(parts[-1]))

        return probabilities

    except Exception as e:
        print(f"Error running IsUnstruct: {e}")
        return []
    finally:
        if os.path.exists(temp_file_path):
            os.remove(temp_file_path)

#### Kreiranje sveobuhvatnog JSON fajla sa svim potrebnim podacima za dalju analizu

In [60]:
import json

def gather_and_store_data_in_batches(verified_cds, record, batch_folder, batch_size=100, start_batch=1):
    os.makedirs(batch_folder, exist_ok=True)
    for batch_idx, i in enumerate(range(0, len(verified_cds), batch_size)):
        if batch_idx + 1 < start_batch:
            continue  # Skip until reaching the desired batch
        batch = verified_cds[i:i+batch_size]
        batch_data = []
        for cds in batch:
            try:
                # Extract basic CDS information
                locus_tag = cds.qualifiers.get('locus_tag', [None])[0]
                protein_id = cds.qualifiers.get('protein_id', [None])[0]
                gene_name = cds.qualifiers.get('gene', [None])[0]
                product = cds.qualifiers.get('product', [None])[0]
                translation = cds.qualifiers.get('translation', [None])[0]
                uniprot_accession = get_cds_accession(cds)
                extracted_cds = cds.extract(record)
                nucleotide_seq = str(extracted_cds.seq)

                # Get disorder probabilities from both algorithms
                iupred3_probabilities = get_disorder_data_iupred3(uniprot_accession, "long")
                isunstruct_probabilities = get_disorder_probabilities_isunstruct(translation)

                # Create CDS data structure
                cds_data = {
                    "locus_tag": locus_tag,
                    "protein_id": protein_id,
                    "gene_name": gene_name,
                    "product": product,
                    "uniprot_accession": uniprot_accession,
                    "location": {
                        "start": int(cds.location.start),
                        "end": int(cds.location.end),
                        "strand": cds.location.strand
                    },
                    "sequences": {
                        "nucleotide": nucleotide_seq,
                        "amino_acid": translation,
                        "length_nt": len(nucleotide_seq),
                        "length_aa": len(translation)
                    },
                    "disorder_analysis": {
                        "iupred3": {
                            "probabilities": iupred3_probabilities,
                            "available": len(iupred3_probabilities) > 0,
                            "disorder_count": sum(1 for p in iupred3_probabilities if p > 0.5) if iupred3_probabilities else 0,
                            "disorder_percentage": round((sum(1 for p in iupred3_probabilities if p > 0.5) / len(iupred3_probabilities) * 100), 2) if iupred3_probabilities else 0
                        },
                        "isunstruct": {
                            "probabilities": isunstruct_probabilities,
                            "available": len(isunstruct_probabilities) > 0,
                            "disorder_count": sum(1 for p in isunstruct_probabilities if p > 0.5) if isunstruct_probabilities else 0,
                            "disorder_percentage": round((sum(1 for p in isunstruct_probabilities if p > 0.5) / len(isunstruct_probabilities) * 100), 2) if isunstruct_probabilities else 0
                        }
                    }
                }
                batch_data.append(cds_data)

            except Exception as e:
                print(f"Neuspešno procesuiran CDS {locus_tag}: {e}")
                continue

        # Save batch
        batch_file = os.path.join(batch_folder, f"batch_{batch_idx+1}.json")
        with open(batch_file, 'w') as f:
            json.dump(batch_data, f, indent=2)
        print(f"Sačuvan {batch_file}")


In [61]:
from datetime import datetime

def merge_batches(batch_folder, final_output_file, verified_cds):
    all_cds = []
    batch_files = sorted([os.path.join(batch_folder, f) for f in os.listdir(batch_folder) if f.endswith('.json')])
    for batch_file in batch_files:
        with open(batch_file) as f:
            all_cds.extend(json.load(f))
    metadata = {
        "genome_accession": "NC_000913.3",
        "total_cds_count": len(verified_cds),
        "created_at": datetime.now().isoformat(),
        "algorithms_used": ["IUPred3", "IsUnstruct"],
        "processed_count": len(all_cds),
        "failed_count": len(verified_cds) - len(all_cds)
    }
    final_data = {
        "metadata": metadata,
        "CDS": all_cds
    }
    with open(final_output_file, 'w') as f:
        json.dump(final_data, f, indent=2)
    print(f"\n=== Finalni JSON Kreiran ===")
    print(f"Uspešno procesuirano: {metadata['processed_count']} CDS-ova")
    print(f"Neuspelo: {metadata['failed_count']} CDS-ova")
    print(f"Izlazni fajl: {final_output_file}")
    print(f"Veličina fajla: {os.path.getsize(final_output_file) / (1024*1024):.2f} MB")

In [ ]:
batch_folder = 'data/batches'
data_to_process = verified_cds
gather_and_store_data_in_batches(data_to_process, record, batch_folder, batch_size=100)
merge_batches(batch_folder, 'data/final_cds_data.json', data_to_process)

Request error for P0DPM5: 500 Server Error:  for url: https://iupred3.elte.hu/iupred3/long/P0DPM5.json
Sačuvan data/batches\batch_1.json
Request error for P0DPM6: 500 Server Error:  for url: https://iupred3.elte.hu/iupred3/long/P0DPM6.json
Request error for P0DSE3: 500 Server Error:  for url: https://iupred3.elte.hu/iupred3/long/P0DSE3.json
Sačuvan data/batches\batch_2.json
Request error for P0DUW0: 500 Server Error:  for url: https://iupred3.elte.hu/iupred3/long/P0DUW0.json
Sačuvan data/batches\batch_3.json
Request error for P0DSE5: 500 Server Error:  for url: https://iupred3.elte.hu/iupred3/long/P0DSE5.json
Sačuvan data/batches\batch_4.json
Sačuvan data/batches\batch_5.json
Sačuvan data/batches\batch_6.json
Request error for P0DSE6: 500 Server Error:  for url: https://iupred3.elte.hu/iupred3/long/P0DSE6.json
Sačuvan data/batches\batch_7.json
